In [2]:

#import pandas as pd
#import time
import datetime
# from tkinter import *

import os.path
import linecache

import tkinter as tk

import mysql.connector as mysql
import pandas as pd
import time
#from datetime import datetime, timedelta
global ProcName





with open('App_Config.csv', "r") as f1:
    lis = [line.split(",") for line in f1]         # create a list of lists
    MasterFileName = lis[0]
    MasterSN = lis[1]    
    MeasPoint = lis[2] 
    MeasIndex = lis[3]
    MeasUnit = lis[4]
    MeasLlim = lis[5]
    MeasClim = lis[6]
    MeasUlim = lis[7]
    
    ResultPath = lis[8]     #,D:\RecData
    ResultExt  = lis[9]     #,B.dat
    CheckReportPath = lis[10]   #,D:\RecData
    StationNo = lis[11]
    IPaddr = lis[12]
    dbTableName = lis[13]
    ModelName = lis[14]
    UnitAdjuster = lis[15] 
    dmProcName = lis[16] 

### For debug -----------

# print("Whole data in line",MasterFileName)
# print("MeasIndex = ",MeasIndex)



# MasterFileName = Configline[1]
fnOKMasterFileName = MasterFileName[1].rstrip()
fnNGMasterFileName = MasterFileName[2].rstrip()

fnMasterSNId = int(MasterSN[1].rstrip())
fnMeasPointRow = int(MeasPoint[1].rstrip())


fnMeasUnit = int(MeasUnit[1].rstrip())
fnMeasLlim = int(MeasLlim[1].rstrip())
fnMeasClim = int(MeasClim[1].rstrip())
fnMeasUlim = int(MeasUlim[1].rstrip())


fnResultPath = ResultPath[1].rstrip()
fnResultExt = ResultExt[1].rstrip()
fnCheckReportPath =  CheckReportPath[1].rstrip()

fnStationNo = StationNo[1].rstrip()
fnIPaddr = IPaddr[1].rstrip()
fndbTableName = dbTableName[1].rstrip()
fnModelName = ModelName[1].rstrip()
fnUnitAdjuster = UnitAdjuster[1].rstrip()
ProcName = dmProcName[1].rstrip()
 
 
print("OK Master file name = " , fnOKMasterFileName)
print("NG Master file name = " , fnNGMasterFileName)

print("ID SN = " , fnMasterSNId)





# Variables Read Test result file ####
# Get current date file name 

resultFileName = datetime.datetime.now().strftime('%Y_%m_%d') + fnResultExt

print("fnResultPath= ", fnResultPath)
print("Result file name = ", resultFileName)

FullResultPath = fnResultPath + "\\" + resultFileName
FullReportPath = fnCheckReportPath + "\\" + fnStationNo + ".txt"

print("Result FULL Path file name = ", FullResultPath,"\n")

print("--RESULT--\n")



## Compare ##
## Common Variables 
OKCount = 0

ReportFileName = "x1930_SCRWH_ST01.txt"




### Read master in DB ##
def ReadMasterDB(masterBarcode):
    global MeasPoint
    global UnitMaster
    global LlimMaster
    global ClimMaster
    global UlimMaster
    global actMasterSN
    global masterData
    global checkMasterType
    
    db = mysql.connect(host = fnIPaddr,user = "YOUR USER NAME",passwd = "YOUR PWD", database = "YOUR DB NAME")
    
    cursor2 = db.cursor()

    ## For debug
    #mCurDate = '2022-08-18' 
    # # Check existing
    sqlStr = "SELECT * FROM refmaster WHERE MasterSN = '" + masterBarcode + "' ORDER BY ID ASC"
    
    # # a = CheckExist(tbName,sqlStr)
    # exist_result = pd.read_sql(sqlStr, db)
    cursor2.execute(sqlStr)
    ## Order of fetchall must before rowcount otherwise rowcount is -1 or wrong
    ## This possible causes program error about updID
    masterData = list(cursor2.fetchall())
    db.close()
    if len(masterData) >= fnMeasPointRow:  
        checkMasterType = masterData[0][11]
    else:
        checkMasterType = "UNKNOWN"
        
    # mResult = cursor2.fetchall()
    # row_count = cursor2.rowcount 
    
    
    
    
    

    #########################################################################    
    ### These variables must be global ###
    # ## Get master data from each line #
    # UnitMaster = lis[fnMeasUnit]
    # LlimMaster = lis[fnMeasLlim]
    # ClimMaster = lis[fnMeasClim]
    # UlimMaster = lis[fnMeasUlim]
    # dmmMasterSN = lis[fnMasterSNId]
    # actMasterSN = dmmMasterSN[1] 
    
    
  
    




def ReadTestResult(dmmFullResultPath):
    # Open lg file ##
    global last_line
    global header_line
    global line
    global DUT_SN
    with open(dmmFullResultPath, "r") as f3:
        # Read the latest line
        last_line1 = f3.readlines()[-1]
    ## if the last line is NULL then read again and get the second last line
    if last_line1 == '\x00':
        with open(dmmFullResultPath, "r") as f4:
            last_line = f4.readlines()[-2]
    else:
        last_line = last_line1
    ## These variables also should be global
    ## Later header should be set in App_Config    
    header_line = linecache.getline(FullResultPath, 5)
        #header_line = f3.readlines()[4]
    line = last_line.split(";")
    print(line)
    DUT_SN = line[fnMasterSNId]

    # # -- For debug purpose --
    # print(len(line))
    # print(type(line))
    # print(line)


    # print("SN = ", line[int(MeasIndex[1])])
    # print("DATE = ", line[int(MeasIndex[2])])
    # print("TIME = ", line[int(MeasIndex[3])])

#######################################################################################

        

# print(MeasPoint[1].rstrip())
# print(LlimMaster[1].rstrip())
# print(ClimMaster[1].rstrip())
# print(UlimMaster[1].rstrip())
# print(line[int(MeasIndex[4])])




def JudgeAndRecord():
    global chkResult
    global OKCount
    global chkResultThai
        
    ## Later adjust file Name depend on model and machine "Model_Process_MachineID"
    ## x1930_SCRWH_ST01
    ## RECORD TO BOTH LOCAL AND SHARED FOLDER ######
    ## LOCAL : SAME folder with exe , SHARED : ReportPath

    ## Check if FILE not exist then create new header file ####################
    fExist = os.path.exists(FullReportPath)
    if not fExist:
        f = open(FullReportPath, "a")
        f.write("EmpNo;" + header_line)
        f.close()

    fExist = os.path.exists(ReportFileName)
    if not fExist:
        f = open(ReportFileName, "a")
        f.write("EmpNo;" + header_line)
        f.close()
    #############################################################################
    
        

    for i in masterData:
        #print(i[1])
        # ## Convert from um to mm  +++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++++
        a = float(line[i[3]])
        a /=float(fnUnitAdjuster)
        #print(a)
        # ###############################################################################
        if float(i[9]) <= a <= float(i[7]):
            OKCount +=1
            chkResult = "PASS"
            
            
        else:
            ## Do nothing : Just fake it 
            OKCount = OKCount
            chkResult = "FAIL"
            
            
        
        print('Point = {0:<10} Llim = {1:<10} Clim = {2:<10} Ulim = {3:<10} Meas. value = {4:<10}  Judgement = {5:<10}'.format(i[1], i[9],i[8],i[7],a,chkResult))
       



    
    
    
    

    
    ### Final judge and Record outside for loop ###
    
    ### Checker + TimeStamp 
      
    #MasterChecker = mEmpNo + " @" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    print("OKcount = ", OKCount)
    #print("MeasPoint = ", len(MeasPoint))
    ## MeasPoint index start from 1 , So TTL meas point should be len()-1 
    
    # ActMeasPoint = len(MeasPoint) - 1
    ActMeasPoint = len(masterData) 
    # print("Len Masterdata = ", len(masterData))
    
    
    if OKCount == ActMeasPoint:
        # OKCount = 0     # RESET COUNTER
        chkResult = "PASS"
        chkResultThai = "ผ่าน"
         ## IF ALL = PASS Record last row to file
        ## Record to LOCAL
        f = open(ReportFileName, "a")
        f.write(mEmpNo + "|" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ";" + last_line)
        f.close()
        ## Record to SHARED
        f = open(FullReportPath, "a")
        f.write(mEmpNo + "|" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S") + ";" + last_line)
        f.close()
    else:
        chkResult = "FAIL"
        chkResultThai = "ไม่ผ่าน"
       
   
       
    
    print(checkMasterType, "confirmation result = ", chkResult)
    
    OKCount = 0     # RESET COUNTER
############# END OF PROCESSING #############################





    
def RecordToDB():
    
   
    
    mCurDate = datetime.datetime.now().strftime('%Y-%m-%d')
    HourCurTime = datetime.datetime.now().strftime('%H')
    
    

    ## Find Col name : Hxx
    if len(HourCurTime) < 2:
        HourName = "0" + HourCurTime    
    else:
        HourName = HourCurTime 
    
    upColName = "H" + HourName


    db = mysql.connect(host = fnIPaddr,user = "YOUR USERNAME",passwd = "YOUR PWD", database = "YOUR DB NAME")
    cursor2 = db.cursor()

    ## For debug
    #mCurDate = '2022-08-18' 
    # # Check existing
    sqlStr = "SELECT * FROM " + fndbTableName + " WHERE inpDate = '" + mCurDate + "' AND StationNo = '" + fnStationNo + "' AND MasterType = '" + checkMasterType + "' ORDER BY ID DESC LIMIT 1"
    
    # # a = CheckExist(tbName,sqlStr)
    # exist_result = pd.read_sql(sqlStr, db)
    cursor2.execute(sqlStr)
    ## Order of fetchall must before rowcount otherwise rowcount is -1 or wrong
    ## This possible causes program error about updID
    
    mResult = cursor2.fetchall()
    row_count = cursor2.rowcount     
    
    #print("DATA MY = ",mResult)
    #print("ROW Count  = ",row_count)
    
    
    # If Not exist then insert draft
    #if (len(exist_result) == 0):
    if row_count == 0:
        
        # Insert ##
        print("NOT EXIST : INSERT", "\n")
        InstStr = "INSERT INTO " + fndbTableName + "(StationNo,ModelName,MasterType,inpDate," + upColName + ")VALUES('" + fnStationNo + "', '" + fnModelName + "' ,'" + checkMasterType + "' , CURDATE(), '" + chkResult + "')"
        print(InstStr)   
        
        result = cursor2.execute(InstStr)    
        db.commit()
    
    else:
        ## Exist 
        print("EXIST : UPDATE", "\n")        
        # Get the last ID
        for r in mResult:
            updID = str(r[0])
        
        
        UpdStr = "UPDATE " + fndbTableName + " SET " + upColName + " = '" + chkResult + "' WHERE id = " + updID    
        print(UpdStr)    
        #cursor2.execute(UpdStr, CnfResult, int(updID))
        cursor2.execute(UpdStr)
        db.commit()
    
    db.close()



## Start GUI tkinter work but later#############################################


#### DISPLAY ##############################################################
#### Print as Table in console ?
#### Show short result on GUI

 
    
    
    
    
        
def FNbutton_clicked():
    global mEmpNo
    mEmpNo = EmpNo.get()
    mEmpNo = mEmpNo.upper()
    global dmmScreenStr
    # actMasterSN = 'GFX113300DZ0WR247'
    
    
    if not mEmpNo == "":
    
        ### Call Read Master file function 
        # ReadMasterFile(useMasterFile)
        ### Call Read Test result function 
        ReadTestResult(FullResultPath)
        ReadMasterDB(DUT_SN)
        
       
        
        if fnMeasPointRow == len(masterData):
            dmmDispStr = "ทดสอบ " + checkMasterType + ": " + DUT_SN
           
            ## CALL FINAL PROCESSING
            JudgeAndRecord()
            
            ## Record into DB ++
            RecordToDB()
            
            
            
            dmmScreenStr = "Tested " + checkMasterType + " by : " + mEmpNo + " @" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            print(dmmScreenStr)
            
            #var.set(chkResult)
            var.set(chkResultThai)
            
            lbl2.config(fg="black")

            if chkResult == "PASS":
                lblShowResult.config(fg="green")
            else:
                lblShowResult.config(fg="red")
                               
            
            
        else:
            # Display Master type    
            dmmDispStr = DUT_SN + " is not Master sample!"            
            # lbl2 = tk.Label(root, text=dmmDispStr, bg="turquoise", fg="red", font=("Times", 12, "bold")).place(x=80, y=145)
            lbl2.config(fg="red")
            var.set('----')
        
    if checkMasterType == 'Master NG':
        lbl2.config(fg="red")
    else:
        lbl2.config(fg="black")
        
    # Display
    var2.set(dmmDispStr)
    dmmScreenStr = "Tested by : " + mEmpNo + " @" + datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    lblFinalSum.config(bg="yellow")
    var3.set(dmmScreenStr)
        
        
  


### NOTES : LATER NO NEED TO CLICK BUTTON
### GET SN FROM THE LAST LINE , THEN FIND THE MASTERTYPE BASED ON SN IN DB

def OKbutton_clicked():
    global checkMasterType
    global useMasterFile
    useMasterFile = fnOKMasterFileName
    # checkMasterType = "OK Master"
    FNbutton_clicked()
    
    
    
    
def NGbutton_clicked():
    global checkMasterType
    global useMasterFile
    useMasterFile = fnNGMasterFileName
    checkMasterType = "NG Master"
    FNbutton_clicked()
    

def func(event):
    # print("You hit return.")
    global mEmpNo
    mEmpNo = EmpNo.get()
    if not mEmpNo == "":
        button1["state"] = "normal"
    else:
        button1["state"] = "disable"



# --- main ---

# - init -
root = tk.Tk()
root.configure(bg="turquoise")
root.geometry("500x300")
root.title("MASTER CONFIRMATION")

# - set columns minimal size -

root.columnconfigure(0, minsize=100)
root.columnconfigure(1, minsize=200)
root.columnconfigure(2, minsize=100)

# - header -

lbl = tk.Label(root, text="Please click START button", bg="turquoise", fg="#00f", font=("Times", 16, "bold italic"))
lbl.grid(column=0, row=0, columnspan=3, sticky='we')



# - menus -


usernameLabel = tk.Label(root, text="EmpNo.").place(x=80, y=35)
cautionLabel = tk.Label(root, text="อย่าลืม Enter").place(x=250, y=35)


# -Process Name #
showProcMsg = tk.StringVar()

showProcMsg.set("Process : " + ProcName)
ProcNameLabel = tk.Label(root, textvariable = showProcMsg,font=("Times", 10, "bold italic"))
ProcNameLabel.place(x=80, y=70)


## Display result label ##
## NOTES : Long command oblect cannot intial completely
## Need to split between create and ( grid , place , etc )
#lblShowResult = tk.Label(root, text="---",width=15, height=3, bg="orange",fg="black",font=("Times", 10, "bold italic")).place(x=80, y=200)
var = tk.StringVar()
var.set('----')

lblShowResult = tk.Label(root, textvariable = var,font=("Times", 50, "bold italic"))
lblShowResult.place(x=80, y=170)

var3 = tk.StringVar()
var3.set('..R.E.A.D.Y..')
lblFinalSum = tk.Label(root, textvariable = var3,font=("Times", 10, "bold italic"))
lblFinalSum.place(x=80, y=260)




## After click button
var2 = tk.StringVar()
var2.set('-- Master --')
lbl2 = tk.Label(root, textvariable = var2, bg="turquoise", fg="black", font=("Times", 12, "bold"))
lbl2.place(x=80, y=145)
EmpNo = tk.StringVar()

usernameEntry = tk.Entry(root, textvariable=EmpNo,font=("Times", 15, "bold"),width=10 ).place(x=130, y=32)
root.bind('<Return>', func)

# - Button 
button1 = tk.Button(text="START", width=15, height=2, bg="green",fg="yellow",font=("Times", 10, "bold italic"),command=OKbutton_clicked)
button1["state"] = "disable"

#button.grid(column=0, row=1)
button1.place(x=80, y=100)



# - start -
root.mainloop()

#################################################################################################################################################


# X1780_scrwh_master_check_v02.py
# Displaying X1780_scrwh_master_check_v02.py.

FileNotFoundError: [Errno 2] No such file or directory: 'App_Config.csv'